In [8]:
# https://huggingface.co/docs/trl/sft_trainer

In [9]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments

# import os
# os.environ['CUDA_VISIBLE_DEVICES']="0"

In [10]:
max_length = 512
MODEL = "EleutherAI/polyglot-ko-1.3b"
# MODEL = "facebook/xglm-564M"
# MODEL = "facebook/xglm-1.7B"
# MODEL = "ai-forever/mGPT"
# MODEL = "maywell/Synatra-42dot-1.3B"

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
)
# model = model.half()

tokenizer = AutoTokenizer.from_pretrained(MODEL)

config.json: 100%|██████████| 738/738 [00:00<00:00, 735kB/s]
c:\Users\kwon\anaconda3\envs\papering\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kwon\.cache\huggingface\hub\models--ai-forever--mGPT. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
pytorch_model.bin: 100%|██████████| 3.45G/3.45G [04

In [11]:
tokenizer.model_max_length = max_length
tokenizer.max_len = max_length

In [12]:
# from datasets import load_dataset
# dataset = load_dataset("csv", data_files="./data/preproc_train.csv", split="train").
# dataset['question']

In [13]:
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
# from peft import LoraConfig

MODEL_PATH = "model/"

DEBUG = False
if DEBUG:
    epochs = 1
    dataset = load_dataset("csv", data_files="./data/preproc_train_debug.csv", split="train")
else:
    epochs = 5
    dataset = load_dataset("csv", data_files="./data/preproc_train.csv", split="train")

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['quenstion'])):
        text = f"""###Q: {example['quenstion'][i]}
###A: {example['answer'][i]}"""
        output_texts.append(text)
    return output_texts

# instruction_template = "###질문:"
response_template = "###A:"
collator = DataCollatorForCompletionOnlyLM(
    # instruction_template=instruction_template,
    response_template=response_template,
    tokenizer=tokenizer)

training_args = TrainingArguments(
    learning_rate=1e-4,
    output_dir=MODEL_PATH + MODEL + "_sft_" + str(epochs),         # output directory to where save model checkpoint
    # evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,
    num_train_epochs=epochs,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=1, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    # per_device_eval_batch_size=16,  # evaluation batch size
    logging_steps=200,             # evaluate, log and save model checkpoints every 1000 step
    # save_steps=50000,
    fp16=True,                      # memory save
    # weight_decay=0.01,
    # warmup_steps=2970,
    #dataloader_num_workers=2,
#     remove_unused_columns=False
    #optim="adafactor",
    # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    save_total_limit=2,           # whether you don't have much space so you let only 3 model weights saved in the disk
#     report_to=False               # when you don't want to log with wandb
)

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    args=training_args,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    max_seq_length=max_length,
)

trainer.train()

Map: 100%|██████████| 6440/6440 [00:00<00:00, 7462.02 examples/s]
c:\Users\kwon\anaconda3\envs\papering\Lib\site-packages\trl\trainer\sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
c:\Users\kwon\anaconda3\envs\papering\Lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
c:\Users\kwon\anaconda3\envs\papering\Lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn("PyTorch is not compiled with NCCL support")


OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 24.00 GiB of which 0 bytes is free. Of the allocated memory 20.59 GiB is allocated by PyTorch, and 752.52 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# import random
# import pandas as pd

# train_df = pd.read_csv('./data/train.csv')
# idx = random.randint(0, 664)
# data = train_df.iloc[1, 1:3]
# ans = train_df.iloc[1, 4]
# q1, q2 = data
# tokened = tokenizer(q1, return_tensors='pt').to(device=f"cuda")
# del tokened['token_type_ids']
# result = model(**tokened)
# result.logits

In [ ]:
import os
model.save_pretrained(os.path.join(MODEL_PATH + MODEL + "_sft_" + str(epochs), "checkpoint-last"))
tokenizer.save_pretrained(os.path.join(MODEL_PATH + MODEL + "_sft_" + str(epochs), "checkpoint-last"))

('model/maywell/Synatra-42dot-1.3B_sft10\\checkpoint-last\\tokenizer_config.json',
 'model/maywell/Synatra-42dot-1.3B_sft10\\checkpoint-last\\special_tokens_map.json',
 'model/maywell/Synatra-42dot-1.3B_sft10\\checkpoint-last\\vocab.json',
 'model/maywell/Synatra-42dot-1.3B_sft10\\checkpoint-last\\merges.txt',
 'model/maywell/Synatra-42dot-1.3B_sft10\\checkpoint-last\\added_tokens.json',
 'model/maywell/Synatra-42dot-1.3B_sft10\\checkpoint-last\\tokenizer.json')